This assumes your have a directory with two sub directories, data and models. You should add to the data directory the data from GITHUB LINK. The results of the finetuning are saved under models. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install `transformers` from master
!pip install transformers
!pip install datasets
#!pip install sentencepiece
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

tokenizers                    0.10.3             
transformers                  4.6.1              


# **Import Necessary Packages**

In [ ]:
from transformers import BertForPreTraining, Trainer, TrainingArguments
from transformers import LineByLineTextDataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForMaskedLM
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling


# **Define Functions**

In [ ]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def create_dataset(tokenizer, train_file, valid_file):
    data = load_dataset("text", data_files={"train": train_file, "validation": valid_file})
    
    def tokenize_function(examples):
        return tokenizer(examples["text"])
    
    tokenized_data = data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
    lm_datasets = tokenized_data.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
    )
    return lm_datasets

In [ ]:
def get_trainer(model_file, train_file, valid_file, model_output_dir):

    model = AutoModelForMaskedLM.from_pretrained(model_file)
    tokenizer = AutoTokenizer.from_pretrained(model_file, do_lower_case=True, use_fast=False)
    lm_datasets = create_dataset(tokenizer, train_file, valid_file)

    print(tokenizer.decode(lm_datasets["train"][1]["input_ids"]))

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

    training_args = TrainingArguments(
        output_dir=model_output_dir,          # output directory
        overwrite_output_dir=True,
        num_train_epochs=3,              # total # of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/logs',            # directory for storing logs
          )

    trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=lm_datasets["train"],
        eval_dataset=lm_datasets["validation"],        # evaluation dataset
        data_collator=data_collator,
      )
    
    return trainer



In [ ]:
### Change to the location of your google drive location
path = '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/'

# **Train Baseline Models**

In [ ]:
model_files = ["dccuchile/bert-base-spanish-wwm-uncased", "mrm8488/RuPERTa-base", # Spanish
               "idb-ita/gilberto-uncased-from-camembert", 'Musixmatch/umberto-wikipedia-uncased-v1', "dbmdz/bert-base-italian-uncased", #Italian
               "bert-base-uncased", "roberta-base", # English
               "hfl/chinese-bert-wwm-ext", "hfl/chinese-roberta-wwm-ext" #Chinese
               ]

train_files = ['/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_train_finetuning_base', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_train_finetuning_base', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_train_finetuning_base', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_train_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_train_finetuning_base', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_train_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_train_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_train_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_train_finetuning_base']

valid_files = ['/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_valid_finetuning_base', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_valid_finetuning_base',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_valid_finetuning_base']

model_output_dirs = ['/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProBETO_BASE', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProRuPERTa_BASE',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProGilBERTo_BASE',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProUmBERTo_BASE', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProITALIAN_BASE', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProBERT_BASE', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProRoBERTa_BASE',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProCHINESE_BASE',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProROCHINESE_BASE']


for x, model_file in enumerate(model_files):

    train_file = train_files[x]
    valid_file = valid_files[x]
    model_output_dir = model_output_dirs[x]
    print(model_output_dir)

    trainer = get_trainer(model_file, train_file, valid_file, model_output_dir)
    trainer.train()
    trainer.save_model(model_output_dir)
    # save tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_file, use_fast=False)
    tokenizer.save_pretrained(model_output_dir)


/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProGilBERTo_BASE


# **Train Adapt Models**

In [ ]:
model_files = ["dccuchile/bert-base-spanish-wwm-uncased", "mrm8488/RuPERTa-base", # Spanish
               "idb-ita/gilberto-uncased-from-camembert", 'Musixmatch/umberto-wikipedia-uncased-v1', "dbmdz/bert-base-italian-uncased", #Italian
               "bert-base-uncased", "roberta-base", # English
               "hfl/chinese-bert-wwm-ext", "hfl/chinese-roberta-wwm-ext" #Chinese
               ]

train_files = ['/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_train_finetuning', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_train_finetuning', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_train_finetuning', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_train_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_train_finetuning', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_train_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_train_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_train_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_train_finetuning']

valid_files = ['/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/es_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/it_valid_finetuning', 
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/en_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_valid_finetuning',
               '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/data/zh_valid_finetuning']

model_output_dirs = ['/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProBETO', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProRuPERTa',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProGilBERTo',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProUmBERTo', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProITALIAN', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProBERT', 
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProRoBERTa',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProCHINESE',
                     '/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProROCHINESE']

for x, model_file in enumerate(model_files):
   
    train_file = train_files[x]
    valid_file = valid_files[x]
    model_output_dir = model_output_dirs[x]
    print(model_output_dir)

    trainer = get_trainer(model_file, train_file, valid_file, model_output_dir)
    trainer.train()
    trainer.save_model(model_output_dir)
    # save tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_file, use_fast=False)
    tokenizer.save_pretrained(model_output_dir)


/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProBETO


Using custom data configuration default-3d564e48e5ff7b41


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-3d564e48e5ff7b41/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.
 





forma resumida un informe del comité de desarrollo económico de eeuu, en el que se destaca la importancia de la investigación básica universitaria americana y ella propone mejorar la enseñanza para asegurar la provisión futura de científicos sobresalientes. [SEP] [CLS] asimismo ella defiende la financiación pública de la investigación básica y él pone de manifiesto que las empresas se centran más en la i + d con objetivos de mercado. [SEP] [CLS] él añadió que el interés de aznar por entrevistar se con los máximos representantes empresariales y sindicales es una muestra de su talante de seguir con el proceso del diálogo social, " eje básico de la política del gobierno y que tan buenos resultados " ha tenido


Step,Training Loss


...

Step,Training Loss


/content/drive/MyDrive/Dissertation/Implicit Causality/Finetune/ProROCHINESE


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default-3df6e5114c2c5af9
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-3df6e5114c2c5af9/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)






的 部 分 情 節 ， 而 能 帶 給 觀 眾 一 些 啟 示 。 」 [SEP] [CLS] 電 視 台 把 一 個 跋 扈 專 橫 、 亂 倫 篡 國 的 武 則 天 演 成 柔 弱 堪 憐 的 女 子 ， 這 是 所 不 能 容 忍 的 。 [SEP] [CLS] 當 時 對 巴 黎 這 個 法 國 首 都 興 奮 不 已 ， 並 決 定 將 各 種 印 象 和 想 像 化 成 音 樂 作 為 表 達 ； 在 巴 黎 譜 寫 此 樂 曲 ， 在 一 次 前 往 維 也 納 行 程 的 途 中 進 行 配 器 工 作 ， 首 演 則


Step,Training Loss
